# Homology and Simplices
### Christina Lee
### Category: Grad

So as I've been trying to learn the pure math side of topology to better understand topological materials, my sources have a lot of theorems, and the examples tend to be esoteric and hard to conceptualize.  There also aren't that many of them.  I almost miss the days back in high school when they forced us to do the same problem 1,000 times over.  

So in order to better understand one of the important topics, simplices and simplicial complexes, I've decided to take the pure math and put them to programming the best that I can.  I don't know if this is the best way of doing things, or if it's useful at all, but I think it helps me understand them.  Hopefully it will help you too.

So what's homology?

Homology is the study of deforming things.  If we push and pull at something, without tearing or glueing, what properties remain the same?  A coffee cup is the same as a donut because they both have one hole.  

We are also the same as a donut because of our digestive track... if you ignore all the open cavities we have inside.  We can at least deform away our lungs.  I'm not an anatomy expert.  

Many of the objects of interest are topologies, which are just collecions of sets that obey certain constraints.  To study these topologies, we can find an equivalent simplicial complex.  We can then compute a variety of things about the simplicial complex that tell us about the properties of the topology.  

Simplicial complexes are made out of simplicies.  

A simplex is an $n$-dimensional generalization of a triangle.  A 0-dim simplex is a point; a 1-dim simplex is a line; 3-dim a tetrahedron; so on and so forth.  

We specify them by an ordering of points, like $P_1 P_2 P_3$. 

We can sum together simplices of dimension $p$ to create a <b>$p$-chain</b>.  Each simpliex in a p-chain has a specific weight that can be either positive or negative.  The ordering of points is equivalent up to the sign of the permutation, so if we put a negative weight onto something, that is the same as permuting two indices.
\begin{equation}
- P_1 P_2 P_3 = - P_3 P_1 P_2 = - P_2 P_3 P_1 = P_2 P_1 P_3 = P_1 P_3 P_2 = P_3 P_2 P_1
\end{equation}

The $p$-chains we can construct from a given simplicial complex tell us something about the structure of the topology.  

In [1]:
type simplex
    s::String 
end

In [2]:
type SimplexComplex
    d::Int
    s::Array{Array{simplex}}
end

In [3]:
type pchain
    p::Int
    S::Array{simplex,1}
    C::Array{Float64,1}
end

In [119]:
# Returns 0 is they two strings are not permutations of each other
# Returns 1 if they are even permutations
# Returns -1 if they are odd permutations

function AreSame(a::String,b::String)
    if length(a) != length(b)
        error("Strings not same length in AreSame")
    end
    
    l=length(a)
    
    x=repmat(collect(1:l),1,l)
    y=transpose(x)-1
    z=(x+y-1)%l+1
    zp=z[end:-1:1,:]

    for jj in 1:l
        n=""
        m=""
        for ii in 1:l
            n=string(n,a[z[ii,jj]]) 
            m=string(m,a[zp[ii,jj]])
        end
        
        if b==n
            return 1
        end
        if b==m
            return -1
        end
        
    end
   
    return 0
end

AreSame (generic function with 1 method)

In [14]:
a=simplex("abcd")
l=4

4

In [121]:
sc

4-element Array{Array{simplex,N},1}:
 simplex[simplex("abcd")]                                                                                                                                                                                                                                                        
 simplex[simplex("bcd"),simplex("cda"),simplex("dab"),simplex("abc")]                                                                                                                                                                                                            
 simplex[simplex("cd"),simplex("db"),simplex("bc"),simplex("da"),simplex("ac"),simplex("cd"),simplex("ab"),simplex("bd"),simplex("da"),simplex("bc"),simplex("ca"),simplex("ab")]                                                                                                
 simplex[simplex("d"),simplex("c"),simplex("b"),simplex("d"),simplex("c"),simplex("b"),simplex("a"),simplex("d"),simplex("c"),simplex("a")  …

In [158]:
starter=simplex("abcd");
l=4
sc=Array{Array{simplex}}(l)
sc[1]=[starter];

lscp=l

lnew=l-1;
lold=l;

for kk=1:3
    
    sclast=sc[kk]
    scnext=Array{simplex}(lscp)
    nsckk=length(sc[kk])
    
    for jj in 1:nsckk
        for ii in 1:lold
            
            scnext[lold*(jj-1)+ii]=simplex(sclast[jj].s[(collect(ii:(ii+lp-1)))%lold+1]) 

        end
    end
    
    sc[kk+1]=scnext;

    lscp=lscp*(lnew)
    
    lold=lnew;
    lnew=lnew-1;
end

In [159]:
for kk in 2:l
    scnow=sc[kk]
    nsckk=length(scnow)

    for jj in 1:(nsckk-1)
        for ii in (jj+1):nsckk
            if 0 != AreSame(scnow[jj].s,scnow[ii].s)
                scnow[jj].s=""
                break
            end
        end
    end

    for jj in (nsckk-1):-1:1
        if scnow[jj].s == ""
            deleteat!(scnow,jj)
        end
    end
end

In [160]:
sc

4-element Array{Array{simplex,N},1}:
 simplex[simplex("abcd")]                                                                    
 simplex[simplex("bcd"),simplex("cda"),simplex("dab"),simplex("abc")]                        
 simplex[simplex("cd"),simplex("bd"),simplex("da"),simplex("bc"),simplex("ca"),simplex("ab")]
 simplex[simplex("d"),simplex("c"),simplex("b"),simplex("a")]                                

In [ ]:
a=simplex("abc")
b=simplex("bcd")
pchain(2,[a,b],[1,1])

In [ ]:
a=pchain(3,["abc"],[1])
b=pchain(3,["bcd"],[1])

In [ ]:
import Base.+
import Base.-

function +(a::pchain,b::pchain)
    if a.p != b.p
           error("Simplexes of different dimension")
    end
    return Simplex(a.p,cat(1,a.S,b.S),cat(1,a.C,b.C))
end

function -(a::pchain)
    return Simplex(a.p,a.S,-a.C)
end

function -(a::pchain,b::pchain)
    if a.p != b.p
        error("Simplexes of different dimension")
    end
    return a+(-b)
end

In [ ]:
a+b

In [ ]:
a-b

In [ ]:
a.S[1]

In [ ]:
string(a.S[1][z[1,1]],a.S[1][z[1,2]])

In [119]:
# Returns 0 is they two strings are not permutations of each other
# Returns 1 if they are even permutations
# Returns -1 if they are odd permutations

function AreSame(a::String,b::String)
    if length(a) != length(b)
        error("Strings not same length in AreSame")
    end
    
    l=length(a)
    
    x=repmat(collect(1:l),1,l)
    y=transpose(x)-1
    z=(x+y-1)%l+1
    zp=z[end:-1:1,:]

    for jj in 1:l
        n=""
        m=""
        for ii in 1:l
            n=string(n,a[z[ii,jj]]) 
            m=string(m,a[zp[ii,jj]])
        end
        
        if b==n
            return 1
        end
        if b==m
            return -1
        end
        
    end
   
    return 0
end

AreSame (generic function with 1 method)

In [ ]:
c=Simplex(3,["abc","bcd","def","bac"],[1,1,1,1])

In [123]:
function Simplify(a::Simplex)

    for ii in 1:length(a.S)
        for jj in (ii+1):length(a.S)

            temp=AreSame(a.S[ii],a.S[jj])
            if temp==1
                a.C[ii]=a.C[ii]+a.C[jj]
                a.C[jj]=0
            elseif temp==-1
                a.C[ii]=a.C[ii]-a.C[jj]
            end
            
        end
    end
    
    for ii in (length(a.S)):-1:1
        if a.C[ii] == 0
            deleteat!(a.C,ii)
            deleteat(a.S,ii)
        end
    end
    
    return a
end

LoadError: UndefVarError: Simplex not defined

In [ ]:
Simplify(c)

In [ ]:
function ∂(a::Simplex)
    if a.p==0
        println("Taking Boundary of Point.")
        d=Simplex(NaN,[],[])
    elseif a.p==1
        b=Array{String}(2*length(a.S))
        C=Array{Float64}(2*length(a.C))
        for ii in 1:length(a.S)
            b[2*ii-1]=a.S[ii][2:2]
            b[2*ii]=a.S[ii][1:1]
            
            C[2*ii-1]=a.C[ii]
            C[2*ii]=-a.C[ii]
        end
        d=Simplex(0,b,C)
    else
        b=Array{String}((a.p+1)*length(a.S))
        C=Array{Float64}((a.p+1)*length(a.S))
        for ii in 1:length(a.S)
            s=a.S[ii]
            for jj in 1:(a.p+1)
                b[(ii-1)*(a.p+1)+jj]=s[(collect(jj:jj+a.p-1)-1)%(a.p+1)+1]
                C[(ii-1)*(a.p+1)+jj]=(-1)^(jj+1)
            end
            println(b)
            println(C)
        end
        d=Simplex(a.p-1,b,C)
    end
    
    return Simplify(d) 
end

In [ ]:
∂(Simplex(3,["abcd"],[1]))

In [ ]:
function ∂helper(A::String)
    l=length(A)
    lp=length(A)-1
    
        d=""
        for ii in 1:l
            if isodd(ii)
                d+=A[(collect(ii:ii+lp-1)-1)%l+1]
            else
                
                d+=-A[(collect(ii:ii+lp-1)-1)%l+1]
            end
        end
        return d=d[2:end]
end

In [ ]:
A="ab"
∂hel(A)


In [ ]:
[[1,-1],[A[2:2],A[1:1]]]

In [ ]:
A[1]